In [1]:
import os
import pandas as pd
from PIL import Image
import requests

In [2]:
data = pd.read_csv("USvideos.csv") #Download from https://www.kaggle.com/datasnaek/youtube-new/version/113#USvideos.csv

In [3]:
data.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [4]:
data.describe()

,category_id,views,likes,dislikes,comment_count
count,34949.000000,3.494900e+04,3.494900e+04,3.494900e+04,3.494900e+04
mean,20.104180,1.880284e+06,6.090286e+04,3.077440e+03,7.017535e+03
std,7.514496,5.756655e+06,1.793106e+05,2.968038e+04,3.061650e+04
min,1.000000,5.490000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,1.994290e+05,4.434000e+03,1.680000e+02,5.210000e+02
50%,24.000000,5.557580e+05,1.499700e+04,5.250000e+02,1.577000e+03
75%,25.000000,1.487513e+06,4.478500e+04,1.591000e+03,4.652000e+03
max,43.000000,1.493761e+08,3.388289e+06,1.674420e+06,1.361580e+06


# Download photos

In [5]:
def getPhoto(dat):
    vId = dat[0]
    link = dat[1]
    if not os.path.exists("data//" + vId + ".jpg"):
        r = requests.get(link)
        with open("data//" + vId + ".jpg", "wb+") as f:
            f.write(r.content)
        return 1
    else:
        return 0

In [6]:
dat = [dat for _, dat in data[["video_id","thumbnail_link"]].iterrows()]

In [7]:
tot = len(dat)
c = 0
d_count = 0
s_count = 0

for _, d in enumerate(dat):
    print("Percent Complete: {:.1f}%".format((c/tot)*100), end="\r")
    
    if getPhoto(d):
        d_count += 1
    else:
        s_count += 1
    c+=1

print("\nFinished")
print(str(d_count), "images downloaded")
print(str(s_count), "images skipped. This could be due to an error in the link or the image\
 may have already been downloaded")

Percent Complete: 100.0%
Finished
164 images downloaded
34785 images skipped. This could be due to an error in the link or the image may have already been downloaded


# Some photos are empty photos (see example below) so we need to get rid of those

![title](assets/imgs/empty.jpg)

In [8]:
def getEmptyImgs():
    empty_imgs = []
    for top, d, files in os.walk("data"):
        tot = len(files)
        c = 0
        for file in files:
            print("Percent Complete: {:.1f}%".format((c/tot)*100), end="\r")
            img_samp = Image.open("data/_suPAag6_i8.jpg")
            img_name = "data" +  "//" + file
            tmp = Image.open(img_name) 

            if tmp == img_samp:
                empty_imgs.append(file)

            c+=1
        break
    print("\nFinished")
    print("{} empty images".format(len(empty_imgs)))
    return empty_imgs

In [9]:
empty = [file.replace(".jpg", "") for file in getEmptyImgs()]

Percent Complete: 100.0%
Finished
164 empty images


# Now we want to clean the Dataframe so that we aren't looking at the videos with empty images

In [10]:
data_cleaned = data.loc[~data["video_id"].isin(empty)]

### Delete those images so we don't read them in later

In [11]:
for file in empty:
    f = "data/" + file + ".jpg"
    try:
        os.remove(f)
    except:
        print("Could not find file {}!".format(f))
    else:
        print(file, "DELETED")

-hg9t3PdhFE DELETED
-QR-TB_k20M DELETED
-xt_j2aIAkg DELETED
0-yiRIWkhLo DELETED
01AEuxSlIMg DELETED
04aPsalAZQI DELETED
07JQ4WZJIbg DELETED
0cZLo3AAan4 DELETED
0DVNtFA-_wA DELETED
0EQuVfP-vXs DELETED
1Apnffv9Yko DELETED
1Nu_oGucpDI DELETED
1qKBNKtzTWg DELETED
1VJixjI_P3s DELETED
2B85lYenaxE DELETED
2omJBSQcm5g DELETED
3ekEfYNQJ1s DELETED
3ILxrRlf3KY DELETED
4BiWtH0HywY DELETED
4eKYR_iL5eU DELETED
4HSul1NBFes DELETED
4sXyLTk0IAQ DELETED
51cbHTKdILA DELETED
5JEI0sjJs8c DELETED
5n32F7mS6Kk DELETED
6JvRVILczps DELETED
6ULZDwevR0M DELETED
6VoJHrdoIJw DELETED
7Bq_H1ci9ec DELETED
7uvxjNut1gk DELETED
7YibRBd46oo DELETED
8Ojf1RVqRn4 DELETED
8WbxbqwdWog DELETED
90uhxbfUfC4 DELETED
9TUBf6l7FBg DELETED
9vlKOFd73XM DELETED
9XSULcLI6Hg DELETED
aAm6AHr14zY DELETED
abeqEEmpbdM DELETED
AEAHRq62p3Q DELETED
ai6NByMmCVU DELETED
aKPz583WRpQ DELETED
A_mlvG_nRsg DELETED
bhoFXkVy8JA DELETED
bJje-B9YLIU DELETED
bLKd4XXUWUo DELETED
BLOIU9DpLq4 DELETED
BLXLPisBe8s DELETED
Bmn2STJUWJs DELETED
bT9HPXKP5dI DELETED


# Now for the fun model stuff

### We'll be using a GAN that looks something like:

![title](assets/imgs/model.jpg)

Our model will take in as input:

- The title of the video

- The channel title

- The category id

- Any tags associated to the video


 - Optionally, it may be interesting to look at including publish time

The model will then output a thumbnail image for us to use for our video

### Prepare the Data!

For us to use the above in our model, we need to think of how to vectorize them:

#### Title

    Since the title is a group of words. Lets first take a deeper look at what we're dealing with.

In [12]:
titles = data_cleaned.title

In [13]:
titles

0                       WE WANT TO TALK ABOUT OUR MARRIAGE
1        The Trump Presidency: Last Week Tonight with J...
2        Racist Superman | Rudy Mancuso, King Bach & Le...
3                         Nickelback Lyrics: Real or Fake?
4                                 I Dare You: GOING BALD!?
5                                    2 Weeks with iPhone X
6                Roy Moore & Jeff Sessions Cold Open - SNL
7                      5 Ice Cream Gadgets put to the Test
8        The Greatest Showman | Official Trailer 2 [HD]...
9        Why the rise of the robots won’t mean the end ...
10       Dion Lewis' 103-Yd Kick Return TD vs. Denver! ...
11       (SPOILERS) 'Shiva Saves the Day' Talked About ...
12              Marshmello - Blocks (Official Music Video)
13                  Which Countries Are About To Collapse?
14                                SHOPPING FOR NEW FISH!!!
15                                        The New SpotMini
16        One Change That Would Make Pacific Rim a Class

In [14]:
titles.describe()

count                                                 34223
unique                                                 5859
top       The Chainsmokers, Drew Love - Somebody (Offici...
freq                                                     20
Name: title, dtype: object

##### Hmmm... that's interesting. There are only 5,859 unique titles? Let's dig deeper, shall we?

In [15]:
data_cleaned.loc[titles == "Nickelback Lyrics: Real or Fake?"]  #Title was included in trending 
                                                                #as of 7/11/2018 at least

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
231,puqaWrEC7tY,17.15.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",772235,15301,1229,3328,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
471,puqaWrEC7tY,17.16.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",827546,15887,1299,3460,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
709,puqaWrEC7tY,17.17.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",863397,16213,1337,2913,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
946,puqaWrEC7tY,17.18.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",885011,16396,1363,2956,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
1174,puqaWrEC7tY,17.19.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",897162,16551,1374,2966,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
1399,puqaWrEC7tY,17.20.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",913268,16729,1386,2988,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...


##### Well that explains that. Since they are trending videos, they can trend for multiple days in a row (or multiple times)

Let's remove any duplicates using the video_id

In [16]:
data_cleaned = data_cleaned.drop_duplicates(subset="video_id")

In [17]:
data_cleaned.loc[titles == "Maroon 5 - Wait"]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
12604,TnCa71iUVxA,18.18.01,Maroon 5 - Wait,Maroon5VEVO,10,2018-01-17T17:00:03.000Z,"Maroon|""Wait""|""222""|""Records/Interscope""|""Reco...",692807,70211,2689,5205,https://i.ytimg.com/vi/TnCa71iUVxA/default.jpg,False,False,False,RED PILL BLUES” is out now.\nhttp://smarturl.i...
16983,4uTNVumfm84,18.09.02,Maroon 5 - Wait,Maroon5VEVO,10,2018-02-08T14:00:00.000Z,"Maroon|""Wait""|""222""|""Records/Interscope""|""Reco...",2455067,197019,3139,11356,https://i.ytimg.com/vi/4uTNVumfm84/default.jpg,False,False,False,RED PILL BLUES” is out now.\nhttp://smarturl.i...


##### That's better. Now let's see that summary again

In [18]:
data_cleaned.describe()

,category_id,views,likes,dislikes,comment_count
count,5770.000000,5.770000e+03,5.770000e+03,5770.000000,5770.000000
mean,20.501560,7.043178e+05,3.179135e+04,1324.864298,4156.372270
std,7.221629,1.853097e+06,1.027891e+05,12278.684441,19935.324153
min,1.000000,5.490000e+02,0.000000e+00,0.000000,0.000000
25%,17.000000,7.712800e+04,1.798500e+03,68.000000,250.000000
50%,24.000000,2.465345e+05,7.472500e+03,223.000000,863.000000
75%,25.000000,6.904878e+05,2.326325e+04,700.000000,2601.000000
max,43.000000,4.843165e+07,2.055137e+06,629120.000000,733373.000000


So deduplicating signicantly decreased our available data to train on. No big deal... hopefully.

*continues exploring*

In [19]:
titles = pd.DataFrame(data_cleaned.title)
titles.describe()
# This isn't that bad given that music videos like Maroon 5's can release 2 sometimes. One initial and then a music video.
# They have different video id's and thumbnails so I'll take it

,title
count,5770
unique,5764
top,Maroon 5 - Wait
freq,2


In [20]:
titles["title length"] = titles.title.apply(len)

In [21]:
def getSentenceLen(sent):
    words = sent.split(" ")
    c = 0
    c += sum([1 for word in words])
    return c

In [22]:
titles["word count"] = titles.title.apply(getSentenceLen)

In [23]:
titles

,title,title length,word count
0,WE WANT TO TALK ABOUT OUR MARRIAGE,34,7
1,The Trump Presidency: Last Week Tonight with J...,62,10
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",53,10
3,Nickelback Lyrics: Real or Fake?,32,5
4,I Dare You: GOING BALD!?,24,5
5,2 Weeks with iPhone X,21,5
6,Roy Moore & Jeff Sessions Cold Open - SNL,41,9
7,5 Ice Cream Gadgets put to the Test,35,8
8,The Greatest Showman | Official Trailer 2 [HD]...,65,12
9,Why the rise of the robots won’t mean the end ...,53,12


In [24]:
titles.describe()

,title length,word count
count,5770.000000,5770.000000
mean,49.531716,8.684922
std,19.583905,3.517462
min,4.000000,1.000000
25%,35.000000,6.000000
50%,47.000000,8.000000
75%,62.000000,11.000000
max,100.000000,24.000000


#### With the above data, we see that the max words our titles have is 24 and the least is 1.
#### Using this, we can pad our titles with something at the end so that they are all the same length
##### Let's use "_"

In [25]:
TITLE_LEN = 24
def padTitle(title):
    titleLength = getSentenceLen(title)
    if  titleLength == 24:
        return title
    else:
        title = title + (" _" * (TITLE_LEN - titleLength))
        return title

In [26]:
titles.title = titles.title.apply(padTitle)
titles["word count"] = titles.title.apply(getSentenceLen)
titles["title length"] = titles.title.apply(len)

In [27]:
titles.describe()

,title length,word count
count,5770.000000,5770.0
mean,80.161872,24.0
std,13.290563,0.0
min,50.000000,24.0
25%,71.000000,24.0
50%,79.000000,24.0
75%,88.000000,24.0
max,124.000000,24.0


In [28]:
data_cleaned.title = titles

##### There we are. Now we can vectorize the words using something like word2vec or gloVe

# -----------------------------------------------------------------------------------------------------------

####  Channel Title

For channel title, we can do something similar to Title

In [29]:
chans = data_cleaned.channel_title

In [30]:
chans.describe()

count     5770
unique    2056
top       ESPN
freq        70
Name: channel_title, dtype: object

#### Unlike video title, we want to keep the repeats in channel title. This is best explained in an example:

If a video by the The Tonight Show is what we're currently looking at, it is probably fair to assume
that Jimmy Fallon will be in the thumbnail.

***Interestingly, ESPN Takes the top spot for most trending videos in our data set***

In [31]:
chans

0                                CaseyNeistat
1                             LastWeekTonight
2                                Rudy Mancuso
3                       Good Mythical Morning
4                                    nigahiga
5                                    iJustine
6                         Saturday Night Live
7                          CrazyRussianHacker
8                            20th Century Fox
9                                         Vox
10                                        NFL
11                                        amc
12                                 marshmello
13                              NowThis World
14                            The king of DIY
15                             BostonDynamics
16                                    Cracked
17                                     TED-Ed
18                                PeterSripol
19                                    SciShow
20                                Life Noggin
21                                

# -----------------------------------------------------------------------------------------------------------

#### Category ID

Luckily enough, we can keep Category ID alone since it is already a nice number

In [32]:
cats = data_cleaned["category_id"]
cats

0        22
1        24
2        23
3        24
4        24
5        28
6        24
7        28
8         1
9        25
10       17
11       24
12       10
13       25
14       15
15       28
16       23
17       27
18       28
19       27
20       27
21       28
22       27
23       26
24       23
25       24
26       28
27       26
28       25
29        1
         ..
34556    27
34558    28
34750    26
34751    24
34753    10
34754    24
34755    25
34757    24
34758    24
34759    28
34760     1
34761    26
34765    19
34766    24
34768    25
34769    24
34770    28
34771    24
34772    24
34774    24
34776    10
34777    24
34781    24
34783    26
34798    17
34800    24
34801    10
34813    10
34816    25
34875    24
Name: category_id, Length: 5770, dtype: int64

# -----------------------------------------------------------------------------------------------------------

#### Tags

We can do almost exactly the same thing with tags that we did with title

In [33]:
tags = pd.DataFrame(data_cleaned.tags)

In [34]:
tags.describe()

,tags
count,5770
unique,5317
top,[none]
freq,239


In [35]:
def getTagCount(s):
    l = s.split("|")
    c = 0
    c += sum([1 for t in l])
    return c

In [36]:
#redefined from above
def getSentenceLen(sent):
    tags = sent.split("|")
    counts = []
    for tag in tags:
        c = 0
        words = tag.split(" ")
        c += sum([1 for word in words])
        counts.append(c)
    return counts

In [37]:
tags["tag count"] = tags.tags.apply(getTagCount)
tags["tag word count"] = tags.tags.apply(getSentenceLen)

In [38]:
print(max(tags["tag word count"]))
print(min(tags["tag word count"]))

[16]
[1]


In [39]:
tags.describe()

,tag count
count,5770.000000
mean,19.789081
std,12.241689
min,1.000000
25%,10.000000
50%,19.000000
75%,29.000000
max,69.000000


In [54]:
TAG_W_COUNT = 16
TAG_COUNT = 69
def padTags(s):
    tags = s.split("|")
    newTags = []
    for tag in tags:
        
        #Pad words
        words = tag.split(" ")
        if len(words) == TAG_W_COUNT:
            newTags.append(" ".join(words))
        else:
            while len(words) < TAG_W_COUNT:
                words.append("-")
            newTags.append(" ".join(words))
        
    while len(newTags) < TAG_COUNT:
        tmp = ""
        for i in range(TAG_W_COUNT-1):
            tmp += "- "
        
        newTags.append(tmp)
            
    return "|".join(newTags)

In [60]:
tags["tags"] = tags.tags.apply(padTags)
tags["tag count"] = tags.tags.apply(getTagCount)
tags["tag word count"] = tags.tags.apply(getSentenceLen)

In [62]:
data_cleaned.tags = tags.tags

In [63]:
data_cleaned

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE_ _ _ _ _ _ ...,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin - - - - - - - - - - - - - -|- ...,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,last week tonight trump presidency - - - - - -...,2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman - - - - - - - - - - - - - -|""r...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?_ _ _ _ _ _ _ ...,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link - - - - - - - - - - - - -|""gmm""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?_ _ _ _ _ _ _ _ _ _ _ ...,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan - - - - - - - - - - - - - - -|""higa"" - - ...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
5,gHZ1Qz0KiKM,17.14.11,2 Weeks with iPhone X_ _ _ _ _ _ _ _ _ _ _ _ _...,iJustine,28,2017-11-13T19:07:23.000Z,"ijustine - - - - - - - - - - - - - - -|""week w...",119180,9763,511,1434,https://i.ytimg.com/vi/gHZ1Qz0KiKM/default.jpg,False,False,False,Using the iPhone for the past two weeks -- her...
6,39idVpFF7NQ,17.14.11,Roy Moore & Jeff Sessions Cold Open - SNL_ _ _...,Saturday Night Live,24,2017-11-12T05:37:17.000Z,"SNL - - - - - - - - - - - - - - -|""Saturday Ni...",2103417,15993,2445,1970,https://i.ytimg.com/vi/39idVpFF7NQ/default.jpg,False,False,False,Embattled Alabama Senate candidate Roy Moore (...
7,nc99ccSXST0,17.14.11,5 Ice Cream Gadgets put to the Test_ _ _ _ _ _...,CrazyRussianHacker,28,2017-11-12T21:50:37.000Z,"5 Ice Cream Gadgets - - - - - - - - - - - -|""I...",817732,23663,778,3432,https://i.ytimg.com/vi/nc99ccSXST0/default.jpg,False,False,False,Ice Cream Pint Combination Lock - http://amzn....
8,jr9QtXwC9vc,17.14.11,The Greatest Showman | Official Trailer 2 [HD]...,20th Century Fox,1,2017-11-13T14:00:23.000Z,"Trailer - - - - - - - - - - - - - - -|""Hugh Ja...",826059,3543,119,340,https://i.ytimg.com/vi/jr9QtXwC9vc/default.jpg,False,False,False,"Inspired by the imagination of P.T. Barnum, Th..."
9,TUmyygCMMGA,17.14.11,Why the rise of the robots won’t mean the end ...,Vox,25,2017-11-13T13:45:16.000Z,"vox.com - - - - - - - - - - - - - - -|""vox"" - ...",256426,12654,1363,2368,https://i.ytimg.com/vi/TUmyygCMMGA/default.jpg,False,False,False,"For now, at least, we have better things to wo..."


# -----------------------------------------------------------------------------------------------------------

In [43]:
import keras
from keras.layers import Conv2D, Activation, Dropout
from keras.models import Sequential

Using TensorFlow backend.
